In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb
sb.set()  # seaborn style plots
from sklearn import linear_model

In [ ]:
ds = pd.read_csv('close_shots_ASPNF2019.txt', delimiter='\s+', index_col='shots')

In [ ]:
print(f'During the ASPNF2019, {len(ds)} GOLEM shots have been performed by 5 groups of students')

# Confinement Time scaling with density

Here we focus on the evolution of the confinement time with respect to the density. The density is got from the equilibrium density signal in GOLEM. The plasma electron confinement time also.

In [ ]:
# rescale the date to more convenient units
ds['tau_e_us'] = 1e6*ds.electron_confinement_time
ds['ne'] = ds.electron_density_equilibrium/1e18
ds2 = ds[['ne', 'tau_e_us']].dropna()

In [ ]:
# fit the data with a robust linear model (which filters the outliers)
lm = linear_model.RANSACRegressor().fit(ds2[['ne']], ds2['tau_e_us']) 
ds2['lin_reg'] = lm.predict(ds2[['ne']])

In [ ]:
# plotting the data and the fitting curve
ax = ds2.plot(kind='scatter', x='ne', y='tau_e_us', ylim=(0,50))
ds2.plot(x='ne', y='lin_reg', color='k', ax=ax, label=None)

ax.set_ylabel(r'Confinement time $\tau_e$ [µs]', fontsize=12)
ax.set_xlabel('Electron Density $n_e$ [$10^{18} m^{-3}$]', fontsize=12)
ax.set_title('ASPNF2019 GOLEM Plasmas')
plt.grid(True)
ax.text(2, 30, r'$\hat \tau_e$'+f'={lm.estimator_.coef_[0]:.2} $\hat n_e$ + {lm.estimator_.intercept_:.2}')


# Fitting the Neo Alcator confinement time scaling

The latter fit as linear density behavior, like in the Neo-Alcator scaling law which is [Goldston, Plasma Phys. Control. Fusion 26 (1984) 87].:
$$
\tau_e = 7.1 \times 10^{-22} \bar n_e a^{1.04} R^{2.04} \sqrt{q(a)}
$$
where
$$
q(a) = \frac{a B_t}{R B_p} \approx \frac{2\pi a^2 B_t}{\mu_0 I_p R_0}
$$

where we have used the plasma current derivation from Maxwell-Ampere:
$$
I_p = 2\pi a B_p / \mu_0
$$



Below we compare the analytical expression with the measured data. We adjusted manually the plasma radisu $a$ of GOLEM in order to fit both data

In [ ]:
from scipy.constants import mu_0
a = 35e-3 # should be ~60e-3 
R0 = 0.4
ds['qa'] = 2 * np.pi * a**2 * ds['toroidal_field_mean'] / (mu_0 * ds['plasma_current_mean']*R0)
ds['tau_e_na'] = 7.1e-22 * ds['electron_density_equilibrium'] * a**1.04 * R0**2.04 * np.sqrt(ds['qa'])

In [ ]:
ax=ds.plot(kind='scatter', x='electron_density_equilibrium', y='electron_confinement_time', label='GOLEM')
ds.plot(kind='scatter', x='electron_density_equilibrium', y='tau_e_na', ax=ax, color='r', label='Neo Alcator ($a$ adjusted)')
ax.set_ylim(0, 50e-6)
ax.set_ylabel(r'Confinement time $\tau_e$ [s]', fontsize=12)
ax.set_xlabel('Electron Density $n_e$ [$m^{-3}$]', fontsize=12)
ax.set_title('ASPNF2019 GOLEM Plasmas')
plt.tight_layout()

Conclusion : the Neo Alcator would fit well assuming a smaller plasma radius of ~35 mm instead of 60 mm as expected...

# How to optimize the plasma duration

In [ ]:
ds.hist('plasma_life', bins=len(ds))

In [ ]:
fig, ax = plt.subplots()
sb.distplot(1e3*ds['plasma_life'].dropna(), kde=False, bins=10, ax=ax)
ax.set_xlabel('plasma life [ms]')

In [ ]:
sub_ds = ds[['tau_e_us', 'plasma_life','tcd','ucd','ub','pressure']].dropna()

Interestingly, the confinement time decreases as the plasma life increases...

In [ ]:
plt.figure()
sb.regplot(x='plasma_life', y='tau_e_us', data=sub_ds)

However, there is clear increase of the confinement time with density (pressure)

In [ ]:
plt.figure()
sb.regplot(x='pressure', y='tau_e_us', data=sub_ds)

In [ ]:
plt.figure()
sb.regplot(x='ucd', y='tau_e_us', data=sub_ds)

The pair plot below tells us that both the confinement time and the plasma duration are relatievly independant of the parameter $\tau_{cd}$

In [ ]:
sb.pairplot(sub_ds, height=2)

In [ ]:
sub_ds[(sub_ds['tcd'] == 0.01)]

What happens if we change $u_{cd}$?

In [ ]:
plt.figure()
sb.regplot(x='ucd', y='plasma_current_mean', data=ds)
sb.regplot(x='ucd', y='plasma_current_mean', data=ds[(ds['pressure_request'] == 20)], label='20 mPa requests')
plt.legend()

In [ ]:
plt.figure()
sb.regplot(x='ucd', y='loop_voltage_mean', data=ds, label='all pulses')
sb.regplot(x='ucd', y='loop_voltage_mean', data=ds[(ds['pressure_request'] == 20)], label='20 mPa requests')
plt.legend()

In [ ]:
plt.figure()
sb.regplot(x='ucd', y='electron_temperature_max', data=ds, label='all pulses')
sb.regplot(x='ucd', y='electron_temperature_max', data=ds[(ds['pressure_request'] == 20)], label='20 mPa requests')
plt.legend()

In [ ]:
plt.figure()
sb.regplot(x='ucd', y='electron_confinement_time', data=ds, label='all pulses')
sb.regplot(x='ucd', y='electron_confinement_time', data=ds[(ds['pressure_request'] == 20)], label='20 mPa requests')
plt.legend()

In [ ]:
plt.figure()
sb.regplot(x='ucd', y='plasma_life', data=ds, label='all pulses')
sb.regplot(x='ucd', y='plasma_life', data=ds[(ds['pressure_request'] == 20)], label='20 mPa requests')
plt.legend()

In [ ]:
plt.figure()
sb.regplot(x='tcd', y='plasma_life', data=ds, label='all pulses')
sb.regplot(x='tcd', y='plasma_life', data=ds[(ds['pressure_request'] == 20)], label='20 mPa requests')
plt.legend()

In [ ]:
plt.figure()
sb.regplot(x='tcd', y='loop_voltage_mean', data=ds, label='all pulses')
sb.regplot(x='tcd', y='loop_voltage_mean', data=ds[(ds['pressure_request'] == 20)], label='20 mPa requests')
plt.legend()

In [ ]:
plt.figure()
sb.regplot(x='tcd', y='plasma_current_mean', data=ds, label='all pulses')
sb.regplot(x='tcd', y='plasma_current_mean', data=ds[(ds['pressure_request'] == 20)], label='20 mPa requests')
plt.legend()